In [11]:
from robustness import model_utils, datasets
DATA = 'ImageNet'
DATA_PATH_DICT = {
    'CIFAR': '/scratch/datasets/cifar10',
    'RestrictedImageNet': '/mnt/nfs/datasets/imagenet-pytorch',
    'ImageNet': '/mnt/nfs/datasets/imagenet-pytorch',
    'H2Z': '/scratch/datasets/A2B/horse2zebra',
    'A2O': '/scratch/datasets/A2B/apple2orange',
    'S2W': '/scratch/datasets/A2B/summer2winter_yosemite',
    'ImageNetAnnot': '/data/theory/robustopt/datasets/imagenet-bounding-boxes'
}
dataset = getattr(datasets, DATA)(DATA_PATH_DICT[DATA])
dataset.num_classes = 10

model_kwargs = {
    'arch': 'resnet50',
    'add_custom_forward': True,
    'dataset': dataset,
    'pytorch_pretrained': False,
    'resume_path': "/mnt/nfs/home/hadi/src/certified-vit/outdir_missingness/cifar/cifar10-resnet50-30epochs/checkpoint.pt.best",
}
net, _ = model_utils.make_and_restore_model(**model_kwargs)
net = net.model

=> loading checkpoint '/mnt/nfs/home/hadi/src/certified-vit/outdir_missingness/cifar/cifar10-resnet50-30epochs/checkpoint.pt.best'
=> loaded checkpoint '/mnt/nfs/home/hadi/src/certified-vit/outdir_missingness/cifar/cifar10-resnet50-30epochs/checkpoint.pt.best' (epoch 26)


In [5]:
getattr(datasets, DATA)

robustness.datasets.CIFAR

In [29]:
import torch as ch
CIFAR_NORMALIZE = torchvision.transforms.Normalize(
    mean=ch.tensor([0.4914, 0.4822, 0.4465]),
    std=ch.tensor([0.2023, 0.1994, 0.2010]),
)
img_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224, 224)),
    CIFAR_NORMALIZE
])


In [30]:
import torchvision
ds = torchvision.datasets.CIFAR10('/mnt/nfs/datasets/cifar', train=False, transform=img_transform)

In [31]:
import torch
dl = torch.utils.data.DataLoader(ds, batch_size=128)

In [61]:
all_val = []
with torch.no_grad():
    for x, y in dl:
        x = x.cuda()
        out = net(x)
        all_val.append((out.argmax(-1) == y.cuda()).cpu())

In [62]:
all_val = torch.cat(all_val)

In [63]:
all_val.float().mean()

tensor(0.9677)

ModuleNotFoundError: No module named 'config'

In [48]:
import sys
sys.path.append('../src')
from ds_utils import get_full_model

In [58]:
prebuilt = get_full_model('deit_small_patch16_224', None)

/data/theory/robustopt/saachij/src/CausalDomainTransfer/run/imagenet/old_deit_small_patch16_224-cd65a155.pth
torchvision models: 
 ['alexnet', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'googlenet', 'inception_v3', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext50_32x4d', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'wide_resnet101_2', 'wide_resnet50_2']
customized models: 
 ['mylinear', 'myvgg', 'myresnet18', 'myresnet50', 'mypreactresnet18', 'mylenet', 'mydensenet121', 'mymobilenet', 'mymobilenet2', 'mydpn92', 'myshufflenet', 'mysenet18', 'mysresnet18', 'mysresnet50']
Vision Transformer models: 
 ['vit_small_patch16_224', 'vit_base_patch16_224', 'vit_b

In [59]:
import torch.nn as nn
prebuilt.head = nn.Linear(prebuilt.head.in_features, 10)

In [60]:
from robustness import model_utils, datasets

dataset = getattr(datasets, DATA)(DATA_PATH_DICT[DATA])
dataset.num_classes = 10

model_kwargs = {
    'arch': prebuilt,
    'add_custom_forward': True,
    'dataset': dataset,
    'pytorch_pretrained': False,
    'resume_path': "/mnt/nfs/home/hadi/src/certified-vit/outdir_missingness/cifar/cifar10-deit_small_patch16_224-30epochs/checkpoint.pt.best",
}
net, _ = model_utils.make_and_restore_model(**model_kwargs)
net = net.model

=> loading checkpoint '/mnt/nfs/home/hadi/src/certified-vit/outdir_missingness/cifar/cifar10-deit_small_patch16_224-30epochs/checkpoint.pt.best'
=> loaded checkpoint '/mnt/nfs/home/hadi/src/certified-vit/outdir_missingness/cifar/cifar10-deit_small_patch16_224-30epochs/checkpoint.pt.best' (epoch 21)
